<div>
    <img src="https://dev.pandas.io/static/img/pandas.svg"><br>
</div>

# 5.0: Pandas

So far we have looked at some fairly simple datasets.  NumPy is great for multi-dimensional arrays, but
book-keeping can be tricky.  Pandas is our friend here.  Pandas adds meta-data to our data, and allows
us to interact with data using names and words, rather than indexes. This can mean that we can
write much clearer code (yay).  It's also really good at working with data that you would have previously
interacted with in spreadsheets.  Spreadsheets are the source of **many** errors, keeping data and
results in the same file is almost criminal! Your data are sacred and should **never be in the same
file that you process them in!**.

Pandas [github README](https://github.com/pandas-dev/pandas/blob/master/README.md) outlines why you should
care about Pandas:

> **pandas** is a Python package providing fast, flexible, and expressive data structures designed to 
make working with "relational" or "labeled" data both easy and intuitive. It aims to be the 
fundamental high-level building block for doing practical, **real world** data analysis in Python.
Additionally, it has the broader goal of becoming the **most powerful and flexible open source 
data analysis / manipulation tool available in any language**. It is already well on its way towards 
this goal.

When Pandas says **real world** think messy data. Measurements of properties of the Earth are *almost always*
messy: data points are missed when power supplies go down, or when it is too wet to get into the field, 
almost all Earth science datasets are noisy, and almost all Earth science data are multi-dimensional and
relational (e.g. multiple variables at one particular place and/or time).  Pandas is really good at coping
with this mess, and **will make your life easier!**

In [1]:
%matplotlib widget

## 5.1: New Zealand Earthquake Catalogue

To explore some of the functionality of Pandas, we need a dataset. One large and freely accesible
geoscience dataset in New Zealand is the GeoNet eatrhquake catalogue. This has hundreds of thousands
of eatrhquakes in it, so should be fun to play around with.

To start off with, we need to get the data.  We could manually query the 
[Quake Search](https://quakesearch.geonet.org.nz/) web-app, but that means we need to
click lots of buttons, and isn't great for just exploring a dataset.  Lets do it
programatically.  We will build a function, but lets look at the steps along the way.

### 5.1.1: Building a query

The Quake Search page can be queried by generating a specific web request in the form:

`"https://quakesearch.geonet.org.nz/csv?bbox={min-longitude},{min-latitude},{max-longitude},{max-latitude}&minmag={min-magnitude}&maxmag={max-magnitude}&mindepth={min-depth}&maxdepth={max-depth}&startdate={start-time}&enddate={end-time}"`

We can build that as a string really easily using variables in place of the curly-brackets things:

In [2]:
format_string = (
    "https://quakesearch.geonet.org.nz/csv?bbox="
    "{min_longitude},{min_latitude},{max_longitude},"
    "{max_latitude}&minmag={min_magnitude}"
    "&maxmag={max_magnitude}&mindepth={min_depth}"
    "&maxdepth={max_depth}&startdate={start_time}"
    "&enddate={end_time}")

min_latitude = -49.0
max_latitude = -40.0
min_longitude = 164.0
max_longitude = 182.0
min_magnitude = 0.0
max_magnitude = 9.0
min_depth = 0.0 # in km
max_depth = 500.0
start_time = "2019-1-1T00:00:00"
end_time = "2020-1-1T00:00:00"

query_string = format_string.format(
    min_latitude=min_latitude,
    max_latitude=max_latitude,
    min_longitude=min_longitude,
    max_longitude=max_longitude,
    min_magnitude=min_magnitude,
    max_magnitude=max_magnitude,
    min_depth=min_depth,
    max_depth=max_depth,
    start_time=start_time,
    end_time=end_time)

print(query_string)

https://quakesearch.geonet.org.nz/csv?bbox=164.0,-49.0,182.0,-40.0&minmag=0.0&maxmag=9.0&mindepth=0.0&maxdepth=500.0&startdate=2019-1-1T00:00:00&enddate=2020-1-1T00:00:00


Great, so we can build a query string simply, and you can see that, because we have used variables in place
of parts of the string, we can change our query really eaisly.  If you click that link it should download
a file called *earthquakes.csv*.

What we really want though is to download that file and look at it in Python straight away.  To do that
we can use the `requests` package to make a web-request:

In [3]:
import requests

response = requests.get(query_string)
print(response)

<Response [200]>


All being well that should say `<Response [200]>`.  200 is the return code to say that all has gone well.

The `Response` object contains the content that we requested from the web as a `.contents` attribute.  Lets have a look at the first 1000 elements of the response:

In [4]:
print(response.content[0:1000])

b'publicid,eventtype,origintime,modificationtime,longitude, latitude, magnitude, depth,magnitudetype,depthtype,evaluationmethod,evaluationstatus,evaluationmode,earthmodel,usedphasecount,usedstationcount,magnitudestationcount,minimumdistance,azimuthalgap,originerror,magnitudeuncertainty\n2019p986237,earthquake,2019-12-31T21:47:06.740Z,2019-12-31T21:49:52.658Z,174.1030579,-41.51368713,1.887827327,24.54598236,M,,LOCSAT,confirmed,manual,iasp91,29,16,5,0.1469085962,91.27532959,0.5454116871,0\n2019p986148,earthquake,2019-12-31T21:00:11.122Z,2019-12-31T21:04:12.241Z,173.2087708,-41.58277893,1.539235603,5.996267319,M,,LOCSAT,confirmed,manual,iasp91,15,10,5,0.2897528708,67.96588135,0.4020879339,0\n2019p986043,earthquake,2019-12-31T20:03:47.465Z,2019-12-31T20:06:20.365Z,176.4427795,-40.39003754,1.668549025,12.87530804,M,,LOCSAT,confirmed,manual,iasp91,26,17,7,0.0738600567,110.4887924,0.5038800067,0\n2019p985959,earthquake,2019-12-31T19:19:04.555Z,2019-12-31T19:21:40.157Z,176.4387512,-40.38579559

This is the contents of the `earthquakes.csv` file.  We can write that to a file in the data directory.  The
contents that we have downloaded are in binary (`print` converted that to a string before
displaying it), so we have to open the file we want to write to using the `wb` argument, which means
"open the file in binary mode with write permission":

In [5]:
with open("data/earthquakes.csv", "wb") as f:
    f.write(response.content)

Now we could read those data in using some convoluted looping and NumPy arrays, or we could
just get Pandas to read it using the 
[pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)
function.  This will quickly parse that large csv file into a Pandas 
[dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html):

In [6]:
import pandas as pd  # It is a normal convention to rename pandas as pd for short

earthquakes = pd.read_csv("data/earthquakes.csv")

print(earthquakes[0:5])

      publicid   eventtype                origintime  \
0  2019p986237  earthquake  2019-12-31T21:47:06.740Z   
1  2019p986148  earthquake  2019-12-31T21:00:11.122Z   
2  2019p986043  earthquake  2019-12-31T20:03:47.465Z   
3  2019p985959  earthquake  2019-12-31T19:19:04.555Z   
4  2019p985713  earthquake  2019-12-31T17:08:01.114Z   

           modificationtime   longitude   latitude   magnitude      depth  \
0  2019-12-31T21:49:52.658Z  174.103058 -41.513687    1.887827  24.545982   
1  2019-12-31T21:04:12.241Z  173.208771 -41.582779    1.539236   5.996267   
2  2019-12-31T20:06:20.365Z  176.442780 -40.390038    1.668549  12.875308   
3  2019-12-31T19:21:40.157Z  176.438751 -40.385796    2.371542  10.608765   
4  2019-12-31T17:22:49.180Z  171.080139 -43.063313    1.901220   5.000000   

  magnitudetype          depthtype  ... evaluationstatus evaluationmode  \
0             M                NaN  ...        confirmed         manual   
1             M                NaN  ...        con

Dataframes are really handy ways of handling "spreadhseet" type data, amongst other things, you can
access columns by their name:

In [7]:
print(earthquakes["origintime"][0:10])

0    2019-12-31T21:47:06.740Z
1    2019-12-31T21:00:11.122Z
2    2019-12-31T20:03:47.465Z
3    2019-12-31T19:19:04.555Z
4    2019-12-31T17:08:01.114Z
5    2019-12-31T16:05:50.502Z
6    2019-12-31T15:47:20.933Z
7    2019-12-31T14:44:43.788Z
8    2019-12-31T14:43:39.777Z
9    2019-12-31T12:37:31.133Z
Name: origintime, dtype: object


And rows by their row number, as we have done above to print just the first five rows of all 
columns, then first first 10 rows of the origintime column.

Each column is a pandas [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html#pandas.Series) which are similar to numpy arrays
and have a lot of the same functionality.

You will note that the time columns (`origintime` and `modificationtime`) have not been
read in as time objects.  We can tell pandas to read those columns in as `datetime` objects
using the `parse_dates` argument.  We can also get rid of the warning about values in column
0 having multiple dtypes by setting the `dtype` argument for the `publicid` column.

In [8]:
earthquakes = pd.read_csv(
    "data/earthquakes.csv", 
    parse_dates=["origintime", "modificationtime"],
    dtype={"publicid": str})

print(earthquakes["origintime"][0:10])

0   2019-12-31 21:47:06.740000+00:00
1   2019-12-31 21:00:11.122000+00:00
2   2019-12-31 20:03:47.465000+00:00
3   2019-12-31 19:19:04.555000+00:00
4   2019-12-31 17:08:01.114000+00:00
5   2019-12-31 16:05:50.502000+00:00
6   2019-12-31 15:47:20.933000+00:00
7   2019-12-31 14:44:43.788000+00:00
8   2019-12-31 14:43:39.777000+00:00
9   2019-12-31 12:37:31.133000+00:00
Name: origintime, dtype: datetime64[ns, UTC]


Now the `dtype` of the `origintime` column is `datetime64`, which is a 64-Bit precision
`datetime` number.

## Aside: Parsing datetimes in Python

We made our query using `str` objects for the start and end-time arguments, but Python has nice native ways of working with dates and times (much nicer than Matlab). These do useful things like cope with leap-years, allow you to add seconds (or other time units) to dates and times, and allow you to format dates and times as `str` objects.

We should switch from giving `str`s to giving `datetime`s.  `datetime` objects come from Python's native `datetime` library, and include a handy `.strftime` method which is literally string-format-time.  We will use that to make the correctly formatted string for our query.  The query requires something of the format:

> year-month-dayThour:minute:second

In `datetime` speak the format string for that is:

> `%Y-%m-%dT%H:%M:%S`

- `%Y` is a four-digit year (e.g.: ..., 2018, 2019, 2020, ...)
- `%m` is a two-digit month (e.g.: 01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12)
- `%d` is a two-digit day (zero-padded as months are, e.g.: 01, 02, ... 28, 29, 30, 31)
- `T` is just a letter, anything not preceded by a `%` sign is interpreted as a `str`
- `%H` is a two-digit hour (as above)
- `%M` is a two-digit minute (as above)
- `%S` is a two-digit second (as above).

Other formatters, for things like day of the week, month as a word, julian-day, milliseconds, etc. can be found in the [offical docs](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes).

Lets see how we could format a `datetime`.  To start we need to make a `datetime` object, we can provide arguments of the year, month, day, hour, minute, second, (millisecond) to make one:

In [9]:
import datetime

# datetime is the module, datetime.datetime is the object itself
test_time = datetime.datetime(2020, 1, 10, 12, 43, 10)
print(test_time)

2020-01-10 12:43:10


Now lets format the string the way that we want it:

In [10]:
print(test_time.strftime("%Y-%m-%dT%H:%M:%S"))

2020-01-10T12:43:10


**Exercise:** format the `datetime` object as "year/month/day hour:minute:seconds".

In [11]:
# Your answer here

Now that we know how it all works, lets put it together into a function with some useful
arguments that we can query.

### 5.1.2 Query using a function

We will set some default values for our arguments, so that we do not always have to
specify every argument.  These defaults are given in the function definition as:

```python
def function(argument=value, ...):
    contents_of_function
```
where `argument` is the argument name, and `value` is the default value for that argument:

In [12]:
def get_geonet_quakes(
    min_latitude=-49.0, max_latitude=-40.0,
    min_longitude=164.0, max_longitude=182.0,
    min_magnitude=0.0, max_magnitude=9.0,
    min_depth=0.0, max_depth=500.0,
    start_time=datetime.datetime(1960, 1, 1),
    end_time=datetime.datetime(2020, 1, 1),
):
    """
    Get a dataframe of the eatrhquakes in the GeoNet catalogue.
    
    Parameters
    ----------
    min_latitude
        Minimum latitude in degrees for search
    max_latitude
        Maximum latitude in degrees for search
    min_longitude
        Minimum longitude in degrees for search
    max_longitude
        Maximum longitude in degrees for search
    min_depth
        Minimum depth in km for search
    max_depth
        Maximum depth in km for search
    min_magnitude
        Minimum magnitude for search
    max_magnitude
        Maximum magnitude for search
    start_time
        Start date and time for search
    end_time
        End date and time for search
        
    Returns
    -------
    pandas.DateFrame of resulting events
    """
    # Convert start_time and end_time to strings
    start_time = start_time.strftime("%Y-%m-%dT%H:%M:%S")
    end_time = end_time.strftime("%Y-%m-%dT%H:%M:%S")
    # Use the more efficient f-string formatting
    query_string = (
        "https://quakesearch.geonet.org.nz/csv?bbox="
        f"{min_longitude},{min_latitude},{max_longitude},"
        f"{max_latitude}&minmag={min_magnitude}"
        f"&maxmag={max_magnitude}&mindepth={min_depth}"
        f"&maxdepth={max_depth}&startdate={start_time}"
        f"&enddate={end_time}")
    print(f"Using query: {query_string}")
    response = requests.get(query_string)
    with open("data/earthquakes.csv", "wb") as f:
        f.write(response.content)
    earthquakes = pd.read_csv(
        "data/earthquakes.csv", 
        parse_dates=["origintime", "modificationtime"],
        dtype={"publicid": str})
    return earthquakes

Lets quickly run this function to get the data.  There won't be any output.  Note that I didn't
provide these data in the repository because:
1. I don't have permission to re-distribute the data and,
2. this dataset gets updated frequently!

In [13]:
earthquakes = get_geonet_quakes(
    start_time=datetime.datetime(2015, 1, 1))

print(earthquakes.columns)

Using query: https://quakesearch.geonet.org.nz/csv?bbox=164.0,-49.0,182.0,-40.0&minmag=0.0&maxmag=9.0&mindepth=0.0&maxdepth=500.0&startdate=2015-01-01T00:00:00&enddate=2020-01-01T00:00:00
Index(['publicid', 'eventtype', 'origintime', 'modificationtime', 'longitude',
       ' latitude', ' magnitude', ' depth', 'magnitudetype', 'depthtype',
       'evaluationmethod', 'evaluationstatus', 'evaluationmode', 'earthmodel',
       'usedphasecount', 'usedstationcount', 'magnitudestationcount',
       'minimumdistance', 'azimuthalgap', 'originerror',
       'magnitudeuncertainty'],
      dtype='object')


That worked, but you might notice that some of the column names have a leading space
in them.  GeoNet doesn't format it's tables particularly nicely, and those leading spaces
are annoying. Lets rename the ` latitude`, ` magnitude` and ` depth` columns without
a leading space.

In [14]:
earthquakes = earthquakes.rename(
    columns={" magnitude": "magnitude",
             " latitude": "latitude",
             " depth": "depth"})

## 5.2: Data visualisation

Now we have a nicely named dataframe, lets have a look at some of the data.
First lets look at magnitude against time. We could use matplotlib directly, but pandas
has some handy plotting shortcuts built in:

In [15]:
import matplotlib.pyplot as plt

earthquakes.plot(x="origintime", y="magnitude", kind="scatter")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Here we specified the `x` argument as the column name we wanted to plot on the x-axis, and
`y` as the other column name.  Pandas has a few different plotting options that can
be specified by the `kind` argument, you can find out more about them 
[here](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html).

You can clearly see the large magnitude Kaikoura earthquake standing out from everything else.

### Exercise:

Pick a specific region based on latitude and longitude ([this website](http://bboxfinder.com/) is
really helpful for finding bounding boxes) and get a dataframe spanning a longer period of
time.  Plot the magnitude vs. time graph for that region.

In [16]:
# Your answer here.  Call your dataframe something different to `earthquakes`

## 5.3: Dataframe statistics

We can quickly get some basic stats from our dataframe, like the median magnitude:

In [17]:
print(earthquakes["magnitude"].median())

2.19577255


As well as lots of other things that you can read about [here](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#computations-descriptive-stats).

### Exercise:

What is the mean, maximum and minimum magnitude in our dataframe?

In [18]:
# Your answer here

## 5.4: Sorting the dateframe

We can easily sort our dataframe as well, lets get an ordered dataframe of earthquakes from
north to south:

In [19]:
earthquakes.sort_values(by=["latitude"], ascending=False)

,publicid,eventtype,origintime,modificationtime,longitude,latitude,magnitude,depth,magnitudetype,depthtype,...,evaluationstatus,evaluationmode,earthmodel,usedphasecount,usedstationcount,magnitudestationcount,minimumdistance,azimuthalgap,originerror,magnitudeuncertainty
15607,2018p089314,earthquake,2018-02-03 01:07:02.723000+00:00,2018-02-05 04:00:18.704000+00:00,176.693573,-40.000092,3.249417,27.374636,M,NaN,...,confirmed,manual,iasp91,67,56,43,0.133015,86.219604,0.645950,0.0
28655,2017p095496,earthquake,2017-02-05 08:00:20.478000+00:00,2017-03-04 07:17:09.734000+00:00,176.230057,-40.000240,3.116431,24.578842,M,NaN,...,confirmed,manual,iasp91,55,49,37,0.089827,31.891479,0.512198,0.0
5623,2019p284688,earthquake,2019-04-16 08:01:55.378000+00:00,2019-04-16 08:11:01.345000+00:00,176.621139,-40.000336,1.497677,35.388756,M,NaN,...,confirmed,manual,iasp91,18,11,7,0.152641,78.319641,0.309052,0.0
7730,2019p036721,earthquake,2019-01-14 14:05:42.206000+00:00,2019-01-14 14:09:52.242000+00:00,176.415314,-40.000515,2.225029,29.730085,M,NaN,...,confirmed,manual,iasp91,17,9,7,0.066559,83.778803,0.403685,0.0
25992,2017p220908,NaN,2017-03-23 17:40:19.481000+00:00,2017-03-23 17:42:58.960000+00:00,173.916725,-40.000536,2.515783,155.937500,M,NaN,...,NaN,automatic,nz3drx,61,61,26,0.802723,165.093863,1.698942,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47660,2016p273370,earthquake,2016-04-11 03:31:22.772000+00:00,2016-05-04 03:25:40.077000+00:00,164.954742,-48.808189,4.893444,33.000000,M,operator assigned,...,confirmed,manual,iasp91,21,14,9,2.844247,309.781241,1.207174,0.0
2041,2019p693230,earthquake,2019-09-14 11:53:12.325000+00:00,2019-10-10 21:47:29.172000+00:00,165.471680,-48.874233,3.594616,33.000000,M,operator assigned,...,confirmed,manual,iasp91,12,7,5,2.655061,331.204990,1.106596,0.0
51230,2015p841124,earthquake,2015-11-08 05:10:30.833000+00:00,2015-11-16 04:31:35.865000+00:00,164.294296,-48.881390,4.430795,33.000000,M,operator assigned,...,confirmed,manual,iasp91,28,14,5,3.161300,333.414967,1.223384,0.0
8205,2018p963742,earthquake,2018-12-23 14:01:23.889000+00:00,2019-01-17 01:06:41.241000+00:00,165.058762,-48.908535,3.809198,33.000000,M,operator assigned,...,confirmed,manual,iasp91,22,13,6,2.864929,329.170319,0.864143,0.0


### Exercise:

Sort the dataframe by depth.

In [20]:
# Your answer here.

## 5.5: Slicing the dataframe

We can also select subsets of our dataframe.  Say you had downloaded the whole catalogue
and realised that you only wanted events shallower than 20km depth, we can do that
with the dataframe:

In [21]:
earthquakes.loc[earthquakes["depth"] <= 20.0]

,publicid,eventtype,origintime,modificationtime,longitude,latitude,magnitude,depth,magnitudetype,depthtype,...,evaluationstatus,evaluationmode,earthmodel,usedphasecount,usedstationcount,magnitudestationcount,minimumdistance,azimuthalgap,originerror,magnitudeuncertainty
1,2019p986148,earthquake,2019-12-31 21:00:11.122000+00:00,2019-12-31 21:04:12.241000+00:00,173.208771,-41.582779,1.539236,5.996267,M,NaN,...,confirmed,manual,iasp91,15,10,5,0.289753,67.965881,0.402088,0.0
2,2019p986043,earthquake,2019-12-31 20:03:47.465000+00:00,2019-12-31 20:06:20.365000+00:00,176.442780,-40.390038,1.668549,12.875308,M,NaN,...,confirmed,manual,iasp91,26,17,7,0.073860,110.488792,0.503880,0.0
3,2019p985959,earthquake,2019-12-31 19:19:04.555000+00:00,2019-12-31 19:21:40.157000+00:00,176.438751,-40.385796,2.371542,10.608765,M,NaN,...,confirmed,manual,iasp91,24,15,6,0.078895,101.663696,0.710030,0.0
4,2019p985713,earthquake,2019-12-31 17:08:01.114000+00:00,2019-12-31 17:22:49.180000+00:00,171.080139,-43.063313,1.901220,5.000000,M,operator assigned,...,confirmed,manual,iasp91,27,19,7,0.176311,105.367616,0.545000,0.0
10,2019p985141,earthquake,2019-12-31 12:03:17.293000+00:00,2019-12-31 12:05:17.430000+00:00,173.130280,-41.642826,1.522212,6.780022,M,NaN,...,confirmed,manual,iasp91,16,9,3,0.206633,75.948624,0.531287,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57966,2015p003202,earthquake,2015-01-02 04:26:27.880000+00:00,2015-01-02 22:04:42.460000+00:00,172.180771,-42.457012,2.403723,5.000000,M,operator assigned,...,confirmed,manual,iasp91,18,16,13,0.331236,125.633993,0.651580,0.0
57970,2015p002370,NaN,2015-01-01 21:03:07.343000+00:00,2015-01-01 21:05:14.259000+00:00,170.371657,-43.471454,2.269890,5.058594,M,NaN,...,NaN,automatic,nz3drx,9,9,7,0.159856,85.300343,0.228440,0.0
57972,2015p002163,NaN,2015-01-01 19:12:54.429000+00:00,2015-01-01 19:14:27.232000+00:00,176.243244,-40.517058,1.720545,19.121094,M,NaN,...,NaN,automatic,nz3drx,17,17,11,0.164357,108.682096,0.252817,0.0
57979,2015p001218,NaN,2015-01-01 10:49:17.507000+00:00,2015-01-01 10:52:25.920000+00:00,174.044606,-41.729973,2.120789,9.804688,M,NaN,...,NaN,automatic,nz3drx,17,17,12,0.126377,109.755740,0.235381,0.0


We can chain multiple conditions together, say we wanted just the earthquakes shallower than 20km and 
greater than magnitude 4:

In [22]:
earthquakes.loc[(earthquakes["depth"] <= 20.0) & (earthquakes["magnitude"] > 4.0)]

,publicid,eventtype,origintime,modificationtime,longitude,latitude,magnitude,depth,magnitudetype,depthtype,...,evaluationstatus,evaluationmode,earthmodel,usedphasecount,usedstationcount,magnitudestationcount,minimumdistance,azimuthalgap,originerror,magnitudeuncertainty
454,2019p922847,earthquake,2019-12-08 10:52:53.590000+00:00,2019-12-18 20:18:35.151000+00:00,174.362320,-41.637001,4.155399,9.677828,M,NaN,...,confirmed,manual,iasp91,40,31,18,0.157785,107.449318,0.432880,0.0
586,2019p901682,earthquake,2019-11-30 14:56:11.982000+00:00,2019-12-12 01:56:15.029000+00:00,171.550659,-43.604828,4.266923,6.307243,M,NaN,...,confirmed,manual,iasp91,49,28,13,0.074215,58.584290,0.351966,0.0
1001,2019p841215,earthquake,2019-11-08 05:59:31.198000+00:00,2019-11-27 01:48:32.066000+00:00,174.740768,-40.637718,4.137776,16.594856,M,NaN,...,confirmed,manual,iasp91,43,36,23,0.257249,76.274231,0.411031,0.0
1213,2019p815147,earthquake,2019-10-29 14:30:52.109000+00:00,2019-11-21 03:31:46.439000+00:00,166.106644,-46.172123,4.213079,12.000000,M,operator assigned,...,confirmed,manual,iasp91,36,24,13,0.398978,282.456741,1.157989,0.0
1232,2019p812026,earthquake,2019-10-28 10:47:21.145000+00:00,2019-10-28 10:52:22.015000+00:00,165.881882,-47.358582,4.267483,5.000000,M,operator assigned,...,confirmed,manual,iasp91,18,15,8,1.313119,314.044306,0.557779,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57778,2015p013973,earthquake,2015-01-06 04:04:57.013000+00:00,2015-01-08 01:54:13.952000+00:00,171.289764,-43.064415,4.829143,5.000000,M,operator assigned,...,confirmed,manual,iasp91,30,21,15,0.358174,107.893917,0.670345,0.0
57808,2015p013444,earthquake,2015-01-05 23:23:40.084000+00:00,2015-01-07 23:01:31.996000+00:00,171.219955,-43.047859,4.673396,5.000000,M,operator assigned,...,confirmed,manual,iasp91,30,23,19,0.355130,113.034483,0.378922,0.0
57886,2015p012836,earthquake,2015-01-05 17:59:52.295000+00:00,2015-01-07 01:38:29.223000+00:00,171.224762,-43.021446,4.117523,5.000000,M,operator assigned,...,confirmed,manual,iasp91,19,17,29,0.337768,117.381105,0.632871,0.0
57890,2015p012824,earthquake,2015-01-05 17:52:48.614000+00:00,2015-01-06 22:47:54.337000+00:00,171.251186,-43.058057,4.157026,5.058594,M,NaN,...,confirmed,manual,nz3drx,17,17,13,0.359664,109.118129,0.542279,0.0


## Exercise:

Select earthquakes deeper than 80km depth between -42 degrees latitude and -44 degrees latitude.

In [23]:
# Your answer here

## 5.6: Do something useful: applying functions to a dataframe

This is all well and good, but the power of programming is in automation.  Lets look at
an example of doing some calculations with a dataframe.  We will use some of what we have
learnt to calculate the occurance rate of earthquakes within a region.  In this case
we will take the region around the top of South Island, containing the faults that ruptured
in the Kaikoura earthquake.

In [53]:
kaikoura = get_geonet_quakes(
    min_latitude=-43.12, max_latitude=-41.15,
    min_longitude=172.37, max_longitude=174.95,
    start_time=datetime.datetime(2010, 1, 1))
# Rename those columns
kaikoura = kaikoura.rename(
    columns={" magnitude": "magnitude",
             " latitude": "latitude",
             " depth": "depth"})

print(f"Downloaded {len(kaikoura)} earthquakes")

Using query: https://quakesearch.geonet.org.nz/csv?bbox=172.37,-43.12,174.95,-41.15&minmag=0.0&maxmag=9.0&mindepth=0.0&maxdepth=500.0&startdate=2010-01-01T00:00:00&enddate=2020-01-01T00:00:00
Downloaded 40825 earthquakes


Earthquake rate is the number of earthquakes per unit time.  For a dataset like this we can calculate
rate as 1 over the inter-event time.

First we need to sort by origin time:

In [54]:
kaikoura = kaikoura.sort_values(by=["origintime"], ignore_index=True)

Then we need to calculate the time between each successive earthquake.  We can do this by
taking the `origintime` column away from a one-sample shifted version of the `origintime` column,
much like we did for calculating the temperature gradient in the DFDP drillhole. In Pandas we
can do this quickly using the `.diff` method:

In [55]:
inter_event_time = kaikoura["origintime"].diff()
print(inter_event_time)

0                   NaT
1       04:49:17.548000
2       02:43:16.490000
3       02:54:41.386000
4       06:41:39.160000
              ...      
40820   00:54:51.599000
40821   01:54:53.833000
40822   02:40:22.484000
40823   06:16:31.345000
40824   00:46:55.618000
Name: origintime, Length: 40825, dtype: timedelta64[ns]


This column is in `timedelta` format. Lets convert it to seconds:

In [56]:
inter_event_time = inter_event_time.dt.total_seconds()
print(inter_event_time)

0              NaN
1        17357.548
2         9796.490
3        10481.386
4        24099.160
           ...    
40820     3291.599
40821     6893.833
40822     9622.484
40823    22591.345
40824     2815.618
Name: origintime, Length: 40825, dtype: float64


The rate is simply 1 / `inter_event_time`:

In [57]:
rate = 1 / inter_event_time

In [58]:
print(rate)

0             NaN
1        0.000058
2        0.000102
3        0.000095
4        0.000041
           ...   
40820    0.000304
40821    0.000145
40822    0.000104
40823    0.000044
40824    0.000355
Name: origintime, Length: 40825, dtype: float64


We can now put this column back into the dataframe and call it `"rate"`:

In [59]:
kaikoura = kaikoura.merge(rate.rename("rate"), left_index=True, right_index=True)

We can plot rate as a function of time. Remember that the time values we use here are the
event origin-times, which do not directly represent the time of the rate calculated, which
is an average between events:

In [60]:
ax = kaikoura.plot(x="origintime", y="rate")
ax.set_ylabel("Instantaneous Rate (earthquakes per second)")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

There is lots more that you can do with dataframes.  Pandas is in heavy use in the datascience
world, and allows you to quickly explore datasets.  Hopefully you will find it useful at some
point in your Geoscience career.

As a final thing, you can save your dataframes:

In [61]:
kaikoura.to_csv("data/kaikoura.csv", index=False)

## 5.7: Extension: computing b-values

Earthquake generally follow a Gutenberg-Richter relationship, where the logarithm of the cumulative number of earthquakes above a given magnitude is proportional to the magnitude:
\begin{equation}
    \log_{10}{N} = a - bM
\end{equation}
where *M* is magnitude, *N* is the number of events with magnitude >= *M*, and *a* and *b* are constants. This is a nice simple straight-line equation with offset from the origin given by *a* and the gradient by *b*.

Some studies (for example, [Nuannin et al., 2005](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2005GL022679)) have found variations in b-value with time and space, and related this to changes in stress.  Lets see if we can:
1. Calculate the b-value for our dataset;
2. Do some sliding-window fu to get at b-value variations in time.

To kick us off, note that in our analysis we are going to miss one fundamental thing which means that everything we do is wrong.  That thing is catalogue completeness, upon which our b-value calculations depend. To show that completeness, and have a first pass at computing b-values, lets look at a cumulative distribution of earthquake magnitudes.

### 5.7.1 Plotting cumulative distributions

We want an inverse cumulative plot of magnitudes. We can do this with matplotlib's `hist` by setting the `cumulative` argument to `-1`, and the `density` argument set to `True`:

In [62]:
fig, ax = plt.subplots()
ax.hist(
    kaikoura["magnitude"], bins=len(kaikoura), 
    histtype="step", density=True, log=True, 
    cumulative=-1)
ax.set_xlabel("Magnitude")
ax.set_ylabel("Cumulative density")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Looks pretty straight on a log-normal plot, as we would expect from the Gutenberg-Richter law.  However, somewhere between M 2 and 3 it stops being straight.  We assume that our catalogue completeness is somewhere in here.  This means that we think that, if we could detect and catalogue all the earthquakes all the way down to the tiny earthquakes, we would continue seeing this log-normal relationship. So, we assume that below our magnitude of completeness ($M_C$) we are missing earthquakes.  This seems reasonable, as earthquakes get smaller they get much harder to detect simply because their amplitudes are greatly reduced.

Lets *assume* our catalogue is complete to $M_C=2.5$ and try and fit a straight line to our cumulative-density plot.

First we will count how many times each magnitude appears in our dataset, we will use a handy object in Pythons native `collections` library, called `Counter`:

In [63]:
from collections import Counter

counted_magnitudes = Counter(kaikoura["magnitude"])

print(counted_magnitudes.most_common(10))  # Print the most common 10 magnitudes

[(2.398, 9), (2.508, 9), (2.23, 9), (2.532, 8), (2.549, 8), (2.459, 7), (2.128, 7), (2.293, 7), (2.147, 7), (2.59, 7)]


Cool, that gives us a list of the magnitude and the number of occurrences of that magnitude.  What we actually want is magnitudes and the number of occurrences of that magnitude and *any magnitude above that magnitude*.  To do that we will:
1. create a unique set of all the magnitudes

In [64]:
magnitudes = set(kaikoura["magnitude"])

2. Make a sorted `list` from this set:

In [65]:
magnitudes = sorted(list(magnitudes), reverse=True)

3. Remove all magnitudes below our completeness using a [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions) (because they are handy):

In [66]:
magnitudes = [m for m in magnitudes if m >= 2.5]

4. Initialise an empty array in which we will put the cumulative density function

In [67]:
import numpy as np

density = np.zeros(len(magnitudes))

5. Loop through the magnitudes from largest to smallest and add the number of occurrences of that magnitude to the total occurrences of the previous magnitude bin:

In [68]:
density[0] = counted_magnitudes[magnitudes[0]]
for i, magnitude in enumerate(magnitudes[1:]):
    density[i + 1] = density[i] + counted_magnitudes[magnitude]

Lets check that that looks okay by plotting it:

In [69]:
fig, ax = plt.subplots()
ax.semilogy(magnitudes, density)
ax.set_ylabel("Cumulative density")
ax.set_xlabel("Magnitude")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Looks good! Now lets try and fit a line to it.  We can use `numpy`'s solvers to do this. Because this is a nice
simple equation we will use the [numpy.polyfit](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) function:

In [70]:
coefficients, residual, rank, singular_values, rcondition = np.polyfit(
    magnitudes, np.log10(density), deg=1, full=True)
b, a = coefficients
print(f"a={a:.2f}, b={b:.2f}")

a=6.27, b=-0.85


b is usually close to 1 (note that the gradient calculated above is negative, which is already taken care of in the Gutenberg-Richter law). 

Lets estimate the density from our calculated values:

In [71]:
# To make our lives easier we will convert our magnitudes to a numpy array:
magnitudes = np.array(magnitudes)
estimated_density = 10 ** (a + (magnitudes * b))

Right, lets see if it fits!

In [72]:
fig, ax = plt.subplots()
ax.semilogy(
    magnitudes, density, marker="+", linestyle="None",
    label="Data")
ax.semilogy(magnitudes, estimated_density, label="Model")
ax.set_ylabel("Cumulative density")
ax.set_xlabel("Magnitude")
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Note that because we specified `full=True` in our call to `polyfit`, we were returned a range of metrics about how well-fitted our data were.  The easiest one of those to understand is the residual:

In [73]:
print(residual)

[7.36381152]


This is a measure of the misfit between our model and our data.

Lets build a simple function to do this with the aim of applying this to distinct time-chunks of our dataset:

In [74]:
def calc_b_value(magnitudes, completeness_magnitude=2.5):
    """
    Calculate the b-value for a range of magnitudes.
    
    Parameters
    ----------
    magnitudes
        List or array of magnitudes
    completeness_magnitude
        Magnitude of completeness for the dataset
        
    Returns
    -------
    b-value
    """
    counted_magnitudes = Counter(magnitudes)
    magnitudes = sorted(list(set(magnitudes)), reverse=True)
    magnitudes = np.array(magnitudes)
    # Remove magnitudes less than completess
    magnitudes = magnitudes[magnitudes >= completeness_magnitude]
    # Calculate density
    density = np.zeros(len(magnitudes))
    density[0] = counted_magnitudes[magnitudes[0]]
    for i, magnitude in enumerate(magnitudes[1:]):
        density[i + 1] = density[i] + counted_magnitudes[magnitude]
    coefficients, residual, rank, singular_values, rcondition = np.polyfit(
        magnitudes, np.log10(density), deg=1, full=True)
    b, a = coefficients
    return b

Lets check that we get the same b-value as we did before:

In [75]:
b = calc_b_value(kaikoura["magnitude"])
print(f"b={b:.2f}")

b=-0.85


### 5.7.2: Rolling windows with Pandas

Pandas has neat ways of doing rolling windows.  We will use this to do two things:
1. Calculate the median date for every 2000 earthquakes;
2. Calculate the b-value for every 2000 earthquakes.

We will then plot these and see if we see any variations.

To calculate the median date we will:
1. sort the dataframe by `"origintime"`
2. Extract just the `"origintime"` and `"magnitude"` columns

In [149]:
window_size = 2000

kaikoura = kaikoura.sort_values(by=["origintime"], ignore_index=True)
magnitude_times = pd.concat([kaikoura["origintime"], kaikoura["magnitude"]], axis=1)

3. Make a new column containing the seconds since the first event - pandas doesn't have a simple way to calculate the median of a range of datetimes, so we will change to working in seconds since a reference time

In [150]:
seconds_offset = (magnitude_times.origintime - magnitude_times.origintime[0]).dt.total_seconds()
magnitude_times = magnitude_times.merge(
    seconds_offset.rename("seconds_offset"), left_index=True, right_index=True)

4. Compute the rolling median of the seconds_offset column

In [151]:
window_median = magnitude_times.seconds_offset.rolling(window_size).median()

5. Convert this column to timedelta objects

In [152]:
window_median = pd.to_timedelta(window_median, unit="S") # Unit is seconds

6. Add the reference time to these to get back to real-time

In [153]:
window_median += magnitude_times.origintime[0]

7. Put this into the dataframe as a new column

In [154]:
magnitude_times = magnitude_times.merge(
    window_median.rename("window_median"), left_index=True, right_index=True)

### Computing moving window b-values

Computing the moving b-value is a little simpler to write, but slower to run.  We will use the function we wrote above and pandas `.rolling().apply(func)` chained method to apply our custom `func` to our column:

In [155]:
b_values = magnitude_times.magnitude.rolling(window_size).apply(calc_b_value)

Lets quickly convert those from gradients to b-values by multiplying by -1:

In [156]:
b_values *= -1

Now we can put those back into the dataframe:

In [157]:
magnitude_times = magnitude_times.merge(
    b_values.rename("b_value"), left_index=True, right_index=True)

### 5.7.4: Plotting our data

Now lets plot it:

In [159]:
ax = magnitude_times.plot(x="window_median", y="b_value", kind="scatter")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 5.7.5: What next?

There are some pretty impressive variations there! In particular there are strong variations in 2013 and 2016, right around when the Cook Strait and Kaikoura earthquakes happened. I wonder if there is anything in that...? **before we get ahead of ourselves**, we missed some key things here that mean that this result is not interpretable:
1. Not all magnitudes are equal, and we were just using GeoNet's summary magnitude;
2. We fixed the magnitude of completeness when in reality completeness depends on a range of factors and is time-varying;
3. We haven't taken spatial variations into account - we have looked at quite a large region here.

We could get around those factors though and extend our rolling window to compute completeness alongside b-value. Potential student project...?

**Exercise:** Using pandas rolling windows, find the mean earthquake location for every window we used above. You will need to compute the rolling mean for latitude, longitude and depth.  Make three plots to show how latitude, longitude and depth vary with time.

In [160]:
# Your answer here

## 5.-1: Summary

That covers *some* of the basics of `dataframes` in pandas.  You should be able to use them to replace most of what you would have done with spreadsheets to allow you to work in a more programatic and reproducible way. We also demonstrated some of the basics of fitting simple polynomials to data.  Note that numpy's `polyfit` function isn't limited to 1st order polynomials, the sky is the limit!

The penultimate things that we are going to cover is some [further plotting](6-More-plotting.ipynb) in the next notebook.